In [1]:
from casadi import *
import matplotlib.pyplot as plt
import numpy as np
import numpy.linalg as la
from tqdm import tqdm

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from Parameters.Parameters_Vaccination_Flat import *




ModuleNotFoundError: No module named 'casadi'

In [ ]:
xdot = vertcat(-u*x[0]*x[1]/N_pop, u*x[0]*x[1]/N_pop - alpha*x[1])
hamiltonian = x[1]**2 - Wu*u**2 + (lbd[1]-lbd[0])*x[0]*x[1]*u/N_pop + (-lbd[1])*alpha*x[1]
s = vertcat(x, lbd)
F = Function('F', [s, u], [vertcat(xdot, -jacobian(hamiltonian, x).T)])
jacobian(hamiltonian, x).shape

In [ ]:
def RK4_Integrator(f, X, U, DT):
       k1 = f(X, U)
       print(k1)
       k2 = f(X + DT/2 * k1, U)
       k3 = f(X + DT/2 * k2, U)
       k4 = f(X + DT * k3, U)
       X=X+DT/6*(k1 +2*k2 +2*k3 +k4)
       return X

In [ ]:

x0 = [N_pop-I0, I0, 0]
M = 50
t = np.linspace(0,T,N)
dt = np.diff(t)[0]/M

x0 = MX.sym('x0', nx)
lbd0 = MX.sym('lbd0', nx)
s0 = vertcat(x0, lbd0)
sk = s0
s_plot = [s0]
lbd_plot = [lbd0]
x_plot = [x0]
for i in range(M):
    sk = RK4_Integrator(F, sk, u, dt)
    lbd_plot.append(sk[0:nx])
    x_plot.append(sk[2:])

s_dot = Function('s_dot', [s0, u], [sk])
f = Function('f_lbd', [x0, lbd0, u], [sk[nx:], sk[nx:]])



In [ ]:
f.sparsity_out(0)

In [ ]:
# "Lift" initial conditions
X0 = MX.sym('x0', nx)
lbd_0 = MX.sym('lbd_0', nx)
Sk = vertcat(X0, lbd_0)
x0 = [N_pop-I0, I0]
w = [Sk]
lbw = [x0]
ubw = [x0]
w0 = [x0]
x_min = [0]*nx
x_max = [N_pop]*nx
lbd_min = [0]*nx
lbd_max = [100]*nx
s_min = x_min + lbd_min
s_max = x_max + lbd_max
Q = []
g = [X0 - x0]
lbg = [0]*nx
ubg = [0]*nx
U = []
lbu = []
ubu = []
U0 = []
u_min = 0.5
u_max = 6.5
u0 = u_max
traj_initial = True
# Formulate the NLP
for k in range(N):
    # New NLP variable for the control
    Uk = MX.sym('U_' + str(k))
    U   += [Uk]
    lbu += [u_min]
    ubu += [u_max]
    U0  += [u0]

    # Integrate till the end of the interval
    Sk_end = F(Sk, Uk)
    
    # New NLP variable for state at end of interval
    if k < (N):
        Sk = MX.sym('S_' + str(k+1), 2*nx)
        w   += [Sk]
        lbw += s_min
        ubw += s_max
    # Add equality constraint
        g   += [Sk_end-Sk]
        lbg += [0]*2*nx
        ubg += [0]*2*nx


g+= [Sk[nx:]]
g = vertcat(*g)
w = vertcat(*w)
U = vertcat(*U)


In [ ]:
from scipy.optimize import fsolve
uk = np.random.uniform(u_min, u_max, N)
f_r = Function('f_r', [w, U], [g])
grad_r = Function('grad_r', [w, U], [jacobian(f_r(w, U), w)])
newton_iter = Function('new_i', [w, U], [w - solve(grad_r(w, U), f_r(w, U))])
G = rootfinder('G', 'kinsol', f_r)

G(np.random.uniform(0, 1000, w.shape[0]), np.random.uniform(u_min, u_max, U.shape))
#G(np.random.uniform(u_min, u_max, w.shape), np.ones(U.shape))


In [ ]:
def Hamiltonian(x, lbd, u):
    return x[1]**2 - Wu*u**2 + (lbd[1]-lbd[0])*x[0]*x[1]*u/N_pop + (lbd[1])*alpha*x[1]

In [ ]:
import os, sys
sys.path.append(os.path.join(os.getcwd(), os.pardir))
from Callbacks.Singleshoot import Singleshoot_CB

lbd0 = [1000,1000]
x0 = [N_pop-I0, I0]
lbd_k = lbd0
tol = 1e-4
rk = DM([tol+1])

X0 = MX.sym('x0', nx)
lbd_0 = MX.sym('lbd_0', nx)
S_sym_k = vertcat(X0, lbd_0)
w = [S_sym_k]
g = [X0-x0]
g.append(lbd_0 - lbd0)
lbw = [0]*2*nx
ubw = [N_pop]*2*nx
xk = x0
lbd_k = lbd0
lbd_f = [tol + 1]*nx
#while la.norm(lbd_f,1) > tol:
U = []

tol = 1e-3
cond = tol + 1
while cond > tol:
    Sk = []
    Sk.append(xk)
    Sk.append(lbd_k)
    U = []
    for k in range(N):
        H1 = Hamiltonian(xk, lbd_k, u_min)
        H2 = Hamiltonian(xk, lbd_k, u_max)
        if H1 < H2:
            uk = u_min
        else:
            uk = u_max
        xk, lbd_k = f(xk,lbd_k,uk)
        Sk.append(xk)
        Sk.append(lbd_k)
        U.append(uk)

    Sk = vertcat(*Sk)
    lbd_f = Sk[-nx:]
    
    Sk = G(Sk, U)
    xk = Sk[:nx]
    lbd_k = Sk[nx:2*nx]
    cond = norm_1(lbd_f)
    print(cond)
    
    
        

In [ ]:
Sk[56:]
Sk[56-nx:56]
f(Sk[56-2*nx:56-nx], Sk[56-nx:56], u_max)


In [ ]:
 # Integrate till the end of the interval
    Sk_end = F(S_sym_k, uk)
    
    if k < (N):
        S_sym_k = MX.sym('S_' + str(k+1), 2*nx)
        w   += [S_sym_k]
        lbw += s_min
        ubw += s_max
    # Add equality constraint
        g   += [Sk_end-S_sym_k]


g+= [S_sym_k[nx:]]
g = vertcat(*g)
w = vertcat(*w)
con = g[:8]
con_lb = [0]*con.shape[0]
con_ub = [0]*con.shape[0]
CB = Singleshoot_CB('Singleshoot_CB', w.shape[0], 0,1,1)   

prob = {'f': norm_2(g), 'x': w, 'g': []}
opts = {}
opts['iteration_callback'] = CB
solver = nlpsol('solver', 'sqpmethod', prob, opts)
sol = solver(x0=vertcat(*Sk), lbx=lbw, ubx=ubw, lbg=[], ubg=[])


In [ ]:
sol['x']

In [ ]:
iter_step = 5
x_sols = [[sol[0::4],sol[1::4]]  for sol in CB.x_sols[0::iter_step]]
lbd_sols = [[sol[2::4],sol[3::4]]  for sol in CB.x_sols[0::iter_step]]
x_sols.append([CB.x_sols[-1][0::4], CB.x_sols[-1][1::4]])
lbd_sols.append([CB.x_sols[-1][2::4], CB.x_sols[-1][3::4]])


import matplotlib.pyplot as plt
from matplotlib import cm
colormap = cm.get_cmap('Greys', len(x_sols))
colors = colormap(np.linspace(.1, .8, len(x_sols)))

fig1, ax1 = plt.subplots(2)
for i, (x_sol, lbd_sol) in enumerate(zip(x_sols, lbd_sols)):
    ax1[0].plot(t, x_sol[0][:-1], color = colors[i])


In [ ]:
plt.plot(Sk)